In [1]:
%matplotlib inline
import openmc

In [2]:
am=openmc.Material(1,"am")

In [3]:
am.add_nuclide('Am242_m1',1)

In [4]:
am.set_density('g/cm3',12.0)
print(am)

Material
	ID             =	1
	Name           =	am
	Temperature    =	None
	Density        =	12.0 [g/cm3]
	S(a,b) Tables  
	Nuclides       
	Am242_m1       =	1            [ao]



In [5]:
graphite=openmc.Material(2,"Be")

In [6]:
graphite.add_nuclide('Be9',1.0)

In [7]:
graphite.set_density('g/cm3',1.85)

In [8]:
vacuummat=openmc.Material(3,"Helium")

In [9]:
vacuummat.set_density('g/cm3',0.001)

In [10]:
vacuummat.add_nuclide('He4',1.0)

In [11]:
mats = openmc.Materials([am,graphite,vacuummat])

In [12]:
mats.export_to_xml()

In [13]:
cyl1=openmc.ZCylinder(R=50)

In [14]:
cyl2=openmc.ZCylinder(R=80,boundary_type='vacuum')

In [15]:
p1=openmc.ZPlane(z0=0)

In [16]:
p2=openmc.ZPlane(z0=3)

In [17]:
p3=openmc.ZPlane(z0=5)

In [18]:
p4=openmc.ZPlane(z0=10)

In [19]:
fuel_region=-cyl1 & +p1 & -p2

In [20]:
moderator_region=-cyl1 & +p3 & -p4

In [21]:
vacuum_region=-cyl2 & +cyl1 | (+p4 & -cyl1) | (-p1 & -cyl1) | (+p2 & -p3 & -cyl1)

In [22]:
fuel=openmc.Cell(1,'fuel')
fuel.fill=am
fuel.region=fuel_region

moderator=openmc.Cell(2,'moderator')
moderator.fill=graphite
moderator.region=moderator_region

vacuum=openmc.Cell(3,'vacuum')
vacuum.fill=vacuummat
vacuum.region=vacuum_region

In [23]:
root = openmc.Universe(cells=(fuel,moderator,vacuum))

In [24]:
geom = openmc.Geometry()
geom.root_universe = root

In [25]:
point = openmc.stats.Point((0, 0, 0.5))
src = openmc.Source(space=point)
geom.export_to_xml()

In [26]:
settings = openmc.Settings()
settings.source = src
settings.batches = 200
settings.inactive = 10
settings.particles = 1000
settings.export_to_xml()

In [27]:
cell_filter = openmc.CellFilter(fuel)

t = openmc.Tally(1)
t.filters = [cell_filter]

In [28]:
t.nuclides = ['Am242_m1']
t.scores = ['total', 'fission', 'absorption', '(n,gamma)']

In [29]:
t2=openmc.Tally(2)
t2.filters = [cell_filter]
t2.scores = ['flux']
tallies = openmc.Tallies([t,t2])
tallies.export_to_xml()

In [30]:
openmc.run()

openmc: Relink `/home/yichengz/anaconda3/bin/../lib/./libgfortran.so.4' with `/lib/x86_64-linux-gnu/librt.so.1' for IFUNC symbol `clock_gettime'

                               %%%%%%%%%%%%%%%
                          %%%%%%%%%%%%%%%%%%%%%%%%
                       %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                     %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                   %%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                ###############      %%%%%%%%%%%%%%%%%%%%%%%%
               ##################     %%%%%%%%%%%%%%%%%%%%%%%
               ###################     %%%%%%%%%%%%%%%%%%%%%%%
               ####################     %%%%%%%%%%%%%%%%%%%%%%
               #####################     %%%%%%%%%%%%%%%%%%%%%
               ######################     %%%%%%%%%%%%%%%%%%%%
               #################

0

In [31]:
!cat tallies.out


 ============================>     TALLY 1     <============================

 Cell 1
   Am242_m1
     Total Reaction Rate                  1.11889        +/- 2.84113E-03
     Fission Rate                         0.285530       +/- 7.34006E-04
     Absorption Rate                      0.300425       +/- 7.95715E-04
     (n,gamma)                            1.50616E-02    +/- 8.50713E-05

 ============================>     TALLY 2     <============================

 Cell 1
   Total Material
     Flux                                 4.65008        +/- 1.12553E-02
